# Intro to profiling

Python's dirty little secret is that it can be made pretty fast.  

The bare-metal HPC people will be angrily tweeting at me now, or rather, they would be if they could get their wireless drivers working.

But we all know that there are some things you *really* don't want to do in Python.  Nested loops are always a bad idea.  But often you won't know where your code is slowing down just by looking at it and trying to accelerate everything can be a waste of time.  

(Developer time, both now and in the future -- you incur technical debt if you unintentionally obfuscate code to make it performant when it doesn't need to be).

The first step is always to find the bottlenecks in your code.

Tools
-----

2. `cProfile`
1. [`line_profiler`](https://github.com/rkern/line_profiler)
3. `timeit`

**Note**:
If you haven't already installed it, you can do

```console
conda install line_profiler
```

or

```console
pip install line_profiler
```

## Some bad code


Here's a bit of code guaranteed to perform poorly.  We can profile it and see where we might be able to help.

In [ ]:
import numpy
from time import sleep

def bad_call(dude):
    sleep(.5)
    
def worse_call(dude):
    sleep(1)
    
def sumulate(foo):
    if not isinstance(foo, int):
        return
    
    a = numpy.random.random((1000, 1000))
    a @ a
    
    ans = 0
    for i in range(foo):
        ans += i
        
    bad_call(ans)
    worse_call(ans)
        
    return ans

In [ ]:
sumulate(150)

## using `cProfile`

cProfile is the builtin profiler in Python.  It provides a function-by-function report of execution time.

In [ ]:
import cProfile

In [ ]:
cProfile.run('sumulate(150)')

## using `line_profiler`

`line_profiler` offers more granular information about each line in a profiled function.

Load the `line_profiler` extension

In [ ]:
%load_ext line_profiler

### For a pop-up window with results in notebook:

Syntax:
`%lprun -f func_to_profile <statement>`

In [ ]:
%lprun -f sumulate sumulate(12)

### Profiling two functions

In [ ]:
%lprun -f bad_call -f worse_call sumulate(13)

### Write results to a text file

In [ ]:
%lprun -T timings.txt -f sumulate sumulate(12)

In [ ]:
%load timings.txt

## `line_profiler` on the command line

Open file, add `@profile` decorator to any function you want to profile

```console
kernprof -l script_to_profile.py
```

will generate `script_to_profile.py.lprof` (pickled result).  To view results run

```console
python -m line_profiler script_to_profile.py.lprof
```

In [ ]:
from IPython.display import IFrame

In [ ]:
IFrame('http://localhost:7000/terminals/1', width=800, height=700)

## `timeit`

`timeit` is not perfect, but it is helpful.  

Potential concerns re: `timeit`

* Returns minimum time of run
* Only runs benchmark 3 times
* It disables garbage collection

```python
python -m timeit -v "print(42)"
```

```python
python -m timeit -r 25 "print(42)"
```

```python
python -m timeit -s "gc.enable()" "print(42)"
```

### Line magic

In [ ]:
%timeit x = 5

### Cell magic

In [ ]:
%%timeit
x = 5
y = 6
x + y

The `-q` flag quiets output.  The `-o` flag allows outputting results to a variable.  The `-q` flag sometimes disagrees with OSX so please remove it if you're having issues.

In [ ]:
a = %timeit -qo x = 5